In [43]:
import requests
import pandas as pd
import json
from dateutil import parser

In [3]:
API_KEY = "4be5daab06c0a1f934a27e5814580751-3352ea772f807c91fd9a5f998493a229"
ACCOUNT_ID = "101-003-29998453-001"
OANDA_URL = "https://api-fxpractice.oanda.com/v3"

In [4]:
session = requests.Session()

In [5]:
session.headers.update({
    "Authorization" : f"Bearer {API_KEY}",
    "Content-Type" : "application/json"
})

In [6]:
params = dict(
    count = 10,
    granularity = "H1"
)

In [7]:
url = f"{OANDA_URL}/accounts/{ACCOUNT_ID}/instruments"

In [8]:
response = session.get(url, params=None, data=None, headers=None)

In [9]:
response.status_code

200

In [10]:
data = response.json()

In [33]:
data = None

In [11]:
instruments_list = data['instruments']

In [12]:
len(instruments_list)

127

In [13]:
instruments_list[0].keys()

dict_keys(['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'minimumTradeSize', 'maximumTrailingStopDistance', 'minimumTrailingStopDistance', 'maximumPositionSize', 'maximumOrderUnits', 'marginRate', 'guaranteedStopLossOrderMode', 'tags', 'financing'])

In [14]:
key_i = ['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision','marginRate']

In [15]:
instruments_dict = {}
for i in instruments_list:
    key = (i['name'])
    instruments_dict[key] = {k : i[k] for k in key_i}

In [16]:
instruments_dict['USD_CAD']

{'name': 'USD_CAD',
 'type': 'CURRENCY',
 'displayName': 'USD/CAD',
 'pipLocation': -4,
 'displayPrecision': 5,
 'tradeUnitsPrecision': 0,
 'marginRate': '0.05'}

In [17]:
with open("../data/insturments.json", "w") as f:
    f.write(json.dumps(instruments_dict, indent=2))

In [79]:
def fetch_candles(pair_name, count=10, granularity="H1"):
    url = f"{OANDA_URL}/instruments/{pair_name}/candles"
    params = dict(
        count = count,
        granularity = granularity,
        price = "MBA"
    )
    response = session.get(url, params=params, data=None, headers=None)
    data = response.json()
    
    if response.status_code == 200:
        if 'candles' not in data:
            data = []
        else:
            data = data['candles']
    return response.status_code, data
        
def get_candles_df(data):
    if len(data) == 0:
        return pd.DataFrame()    
    
    prices = ['mid', 'bid', 'ask']
    ohlc = ['o', 'h', 'l', 'c']

    final_data = []
    for candle in data:
        if candle['complete'] == False:
            continue
        new_dict = {}
        new_dict['time'] = parser.parse(candle['time'])
        new_dict['volume'] = candle['volume']
        
        for p in prices:
            if p in candle:  # Check if the price type exists in the candle
                for o in ohlc:
                    new_dict[f"{p}_{o}"] = float(candle[p][o])  # Access the price data safely
            else:
                # If the price type is missing, skip or set default values
                for o in ohlc:
                    new_dict[f"{p}_{o}"] = None  # Set to None or a default value

        final_data.append(new_dict)
    df = pd.DataFrame.from_dict(final_data)
    return df

def create_data_file(pair_name, count=10, granularity="H1"):
    code, data = fetch_candles(pair_name, count, granularity)
    if code != 200:
        print("Failed", pair_name, data)
        return
    if len(data) == 0:
        print("No candles", pair_name)
    candles_df = get_candles_df(data)
    candles_df.to_pickle(f"../data/{pair_name}_{granularity}.pkl")
    print(f"{pair_name} {granularity} {candles_df.shape[0]} candles, {candles_df.time.min()} {candles_df.time.max()}")


In [76]:
code, data = fetch_candles('EUR_USD', count=10, granularity="H4")
candles_df = get_candles_df(data=data)

In [80]:
create_data_file("EUR_USD", count=10, granularity="H4")

EUR_USD H4 9 candles, 2024-10-23 17:00:00+00:00 2024-10-25 01:00:00+00:00


In [85]:
our_curr = ['EUR', 'USD', 'GBP', 'JPY', 'CHF', 'NZD', 'CAD', 'AUD']

for p1 in our_curr:
    for p2 in our_curr:
        pr = f"{p1}_{p2}"
        if pr in instruments_dict:
            for g in ["H1", "H4"]:
                create_data_file(pr, count=4001, granularity=g)

EUR_USD H1 4000 candles, 2024-03-06 12:00:00+00:00 2024-10-25 04:00:00+00:00
EUR_USD H4 4000 candles, 2022-04-01 13:00:00+00:00 2024-10-25 01:00:00+00:00
EUR_GBP H1 4000 candles, 2024-03-06 12:00:00+00:00 2024-10-25 04:00:00+00:00
EUR_GBP H4 4000 candles, 2022-04-01 13:00:00+00:00 2024-10-25 01:00:00+00:00
EUR_JPY H1 4000 candles, 2024-03-06 12:00:00+00:00 2024-10-25 04:00:00+00:00
EUR_JPY H4 4000 candles, 2022-04-01 05:00:00+00:00 2024-10-25 01:00:00+00:00
EUR_CHF H1 4000 candles, 2024-03-06 12:00:00+00:00 2024-10-25 04:00:00+00:00
EUR_CHF H4 4000 candles, 2022-04-01 13:00:00+00:00 2024-10-25 01:00:00+00:00
EUR_NZD H1 4000 candles, 2024-03-06 12:00:00+00:00 2024-10-25 04:00:00+00:00
EUR_NZD H4 4000 candles, 2022-04-03 21:00:00+00:00 2024-10-25 01:00:00+00:00
EUR_CAD H1 4000 candles, 2024-03-06 12:00:00+00:00 2024-10-25 04:00:00+00:00
EUR_CAD H4 4000 candles, 2022-04-01 13:00:00+00:00 2024-10-25 01:00:00+00:00
EUR_AUD H1 4000 candles, 2024-03-06 12:00:00+00:00 2024-10-25 04:00:00+00:00